## **Assignment 1**

In [1]:
import numpy as np
import pandas as pd
import io
import math
import cv2
import os
import glob
from scipy import linalg
from numpy.linalg import svd
import scipy
import mpmath

In [2]:
def adc(data): #gives Average Distance and Centre
  avg_dist = 0
  num_points = data.shape[0]
  centre = np.mean(data, axis=0)
  for x in data:
    avg_dist += np.linalg.norm(x-centre)
  avg_dist/=num_points
  return avg_dist, centre

In [3]:
def normalize(data, value):

  def norm_matrix(data, value):
    dist, centre = adc(data)
    if data.shape[1] == 2:
      T = np.array([[1, 0, -1*centre[0]], [0, 1, -1*centre[1]], [0, 0, 1]])
    else:
      T = np.array([[1, 0, 0, -1*centre[0]], [0, 1, 0, -1*centre[1]], [0, 0, 1, -1*centre[2]], [0, 0, 0, 1]])
    T*=value/dist
    v = data.shape[1]
    T[v, v] = 1
    return T

  def normalized_coord(data, value):
    T = norm_matrix(data, value)
    num_points = data.shape[0]
    a1 = [np.ones(num_points)]
    coord = np.append(data.T, a1, axis = 0)
    xnew = np.dot(T, coord)
    c2 = np.mean(data, axis=0)
    normalised_coord = xnew[0:data.shape[1]].T
    return normalised_coord

  T = norm_matrix(data, value)
  datan = normalized_coord(data, value)
  return T, datan

In [55]:
def estimate_P(x, X, T, U):
  num_points = X.shape[0]
  X = np.append(X, np.ones((num_points, 1)), axis = 1)
  x = np.append(x, np.ones((num_points, 1)), axis = 1)
  ax = np.append(-X,np.zeros((X.shape[0],4)), axis = 1)
  ax = np.append(ax,x[:,0][:, np.newaxis]*X, axis = 1)
  ay = np.append(np.zeros((X.shape[0],4)),-X, axis = 1)
  ay = np.append(ay,x[:,1][:, np.newaxis]*X, axis = 1)
  M = np.append(ax,ay, axis = 1).reshape(2*X.shape[0],12)
  M = -1*M
  u, s, vh = svd(M)
  v = vh.T
  p = v[:,-1]
  Pn = p.reshape(3,4)
  P = np.dot(Pn, U)
  P = np.dot(np.linalg.inv(T), P)
  return Pn, P

In [5]:
def parameters(P):
  m = P[:,0:3]
  K, R = scipy.linalg.rq(m) 
  X0 = -np.linalg.inv(m)@P[:,-1]
  return K, R, X0

In [6]:
def predict(P, X):
  Xtest = np.append(X, np.ones((X.shape[0],1)), axis=1)
  xtest = P@Xtest.T
  xtest = xtest/xtest[2]
  xtest = xtest[0:2].T
  return xtest

In [7]:
def get_rmse(xtest, x):
    return np.sqrt(np.mean((xtest-x)**2))

In [56]:
def get_parameters(P):
    A = P[:,0:3]
    b = P[:,-1]
    epsilon = -1
    rho = epsilon/np.linalg.norm(A[2])
    x0 = rho*rho*np.dot(A[0],A[2])
    y0 = rho*rho*np.dot(A[1],A[2])
    a1_a3 = np.cross(A[0],A[2])
    a2_a3 = np.cross(A[1],A[2])
    cos_theta = (-1)*np.dot(a1_a3, a2_a3)/(np.linalg.norm(a1_a3)*np.linalg.norm(a2_a3))
    theta = math.acos(cos_theta)*180/math.pi
    sin_theta = math.sin(theta*math.pi/180)
    alpha = rho*rho*np.linalg.norm(a1_a3)*sin_theta
    beta = rho*rho*np.linalg.norm(a2_a3)*sin_theta
    K = np.array([[alpha, (-1)*alpha*cos_theta/sin_theta, x0], [0, beta/sin_theta, y0], [0, 0, 1]])   
    r3 = rho*A[2]
    r1 = a2_a3/np.linalg.norm(a2_a3)
    r2 = np.cross(r3, r1)
    R = np.empty((0,3))
    R = np.append(R, r1.reshape((1,3)), axis=0)
    R = np.append(R, r2.reshape((1,3)), axis=0)
    R = np.append(R, r3.reshape((1,3)), axis=0)
    t = rho*np.dot(np.linalg.inv(K), b)   
    print('K:', K, '\n')
    print('R:', R, '\n')
    print('t:', t,'\n')
    return alpha, beta, theta, rho, x0, y0, K, R, t

# Upload file

In [9]:
X = np.array([[0, 2, 2], [0, 2, 1], [0, 2, 0],[1, 2, 0],[2, 2, 0],[0, 1, 2],[0, 1, 1],[0, 1, 0],[1, 1, 0],[2, 1, 0], [0, 0, 2],[0, 0, 1],[0, 0, 0],[1, 0, 0],[2, 0, 0]])

In [10]:
from google.colab import files
uploaded = files.upload()
x1 = np.loadtxt('img1.txt')
x2 = np.loadtxt('img2.txt')
x3 = np.loadtxt('img3.txt')
x4 = np.loadtxt('img4.txt')
x5 = np.loadtxt('img5.txt')
x6 = np.loadtxt('img6.txt')
x7 = np.loadtxt('img7.txt')
x8 = np.loadtxt('img8.txt')
x9 = np.loadtxt('img9.txt')
x10 = np.loadtxt('img10.txt')
x11 = np.loadtxt('img11.txt')
x12 = np.loadtxt('img12.txt')
x13 = np.loadtxt('img13.txt')
x14 = np.loadtxt('img14.txt')
x15 = np.loadtxt('img15.txt')

Saving img1.txt to img1 (4).txt
Saving img2.txt to img2 (2).txt
Saving img3.txt to img3 (2).txt
Saving img4.txt to img4 (2).txt
Saving img5.txt to img5 (2).txt
Saving img6.txt to img6 (2).txt
Saving img7.txt to img7 (2).txt
Saving img8.txt to img8 (2).txt
Saving img9.txt to img9 (2).txt
Saving img10.txt to img10 (2).txt
Saving img11.txt to img11 (2).txt
Saving img12.txt to img12 (2).txt
Saving img13.txt to img13 (2).txt
Saving img14.txt to img14 (2).txt
Saving img15.txt to img15 (2).txt


In [11]:
x = (x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15)

In [57]:
rmse, alpha, beta, theta, rho, x0, y0 = [0]*15, [0]*15, [0]*15, [0]*15, [0]*15, [0]*15, [0]*15

In [58]:
for i in range(len(x)):
  print('Performing camera calibration for Image', i+1)
  T, xn = normalize(x[i], math.sqrt(2))
  U, Xn = normalize(X, math.sqrt(3))
  Pn, P = estimate_P(xn, Xn, T, U)
  K, R, X0 = parameters(P)
  xtest = predict(P, X)
  rmse[i] = get_rmse(xtest, x[i])
  alpha[i], beta[i], theta[i], rho[i], x0[i], y0[i], K, R, t = get_parameters(P)

Performing camera calibration for Image 1
K: [[1.10338122e+03 6.18121324e+00 2.87611056e+02]
 [0.00000000e+00 1.08687346e+03 5.92085512e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] 

R: [[ 0.52417528 -0.21526337 -0.82395507]
 [ 0.08674089  0.97598946 -0.19980137]
 [ 0.84718138  0.03326034  0.53026169]] 

t: [ -5.91121212   0.62483756 -18.69328709] 

Performing camera calibration for Image 2
K: [[1.05773353e+03 1.48980133e+01 2.29424806e+02]
 [0.00000000e+00 1.07507580e+03 7.04215704e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] 

R: [[ 0.38089299 -0.42282118 -0.82227902]
 [ 0.09563512  0.90256236 -0.41980365]
 [ 0.91965997  0.08126151  0.38421623]] 

t: [ -8.54710566   3.36008899 -20.86681714] 

Performing camera calibration for Image 3
K: [[1.17486128e+03 5.15176069e+01 6.69563861e+02]
 [0.00000000e+00 1.09651871e+03 5.08964772e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] 

R: [[ 0.80002493  0.10583998 -0.59055738]
 [ 0.09049419 -0.99434291 -0.05561462]
 [-0.

In [59]:
def averaged_parameters():

  def avg(x):
    s = 0
    for i in range(len(x)):
      s+=x[i]
    return s/len(x)
  error = avg(rmse)
  i1 = avg(alpha)
  i2 = avg(beta)
  i3 = avg(theta)
  i4 = avg(x0)
  i5 = avg(y0)
  e1 = avg(rho)
  print('Averaged RMSE error :', error)
  print('alpha:', i1)
  print('beta:', i2)
  print('theta:', i3)
  print('x0:', i4)
  print('y0:', i5)
  print('rho (scaling factor):', e1)

In [60]:
averaged_parameters()

Averaged RMSE error : 1.3948199373250298
alpha: 1261.96579722155
beta: 1199.7816470536552
theta: 91.44595547525287
x0: 514.5626595250728
y0: 412.9328558542285
rho (scaling factor): -34.82051986697659
